In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SequentialFeatureSelector

from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

In [2]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_row", 160)
pd.set_option('max_colwidth', 200)

In [3]:
data = pd.read_csv('../../Data/20x42_D66015.csv')

In [4]:
data.head(5)

,loan_amnt,annual_inc,dti,fico_range_low,fico_range_high,int_rate,installment,open_acc,total_acc,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,acc_now_delinq,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,pub_rec,months_cr_line,term,initial_list_status,sub_grade,emp_length,MORTGAGE,OWN,RENT,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,moving,other,small_business,vacation,Not Verified,Source Verified,Verified,loan_status_bin
0,-1.617892,-0.159444,0.818226,-0.445124,-0.445124,-0.652128,-1.618351,-0.470515,0.027112,0.040613,0.258267,-0.984507,-0.48967,0.0,-0.132686,-0.212846,-0.636643,-0.352472,-0.152326,-0.058926,0.942809,-1.429022,-0.019978,-0.727393,-0.371208,0.936282,-0.208138,-0.321019,-0.732963,-0.257722,-0.11847,-0.198918,-0.225630,-0.208138,1.659291,-0.11847,-0.179284,1.212471,-0.738549,-0.565362,0.0
1,0.998307,-0.013596,-0.021126,-0.756603,-0.756603,1.344381,1.047605,0.368860,0.119291,-0.434052,-0.718750,-0.106351,-0.48967,0.0,-0.132686,-0.212846,-0.633592,3.160097,0.506178,-0.058926,0.942809,0.699779,-1.669549,-0.727393,-0.371208,0.936282,-0.208138,-0.321019,-0.732963,3.880149,-0.11847,-0.198918,-0.225630,-0.208138,-0.602667,-0.11847,-0.179284,-0.824762,1.354006,-0.565362,1.0
2,-0.496664,0.245284,0.616782,-0.445124,-0.445124,1.344381,-0.463143,-0.680359,-1.079039,1.434940,1.718384,-0.984507,-0.48967,0.0,-0.132686,-0.212846,-0.416120,-0.352472,2.749971,-0.058926,0.942809,0.699779,-0.294906,-0.727393,2.693908,-1.068055,-0.208138,-0.321019,-0.732963,-0.257722,-0.11847,-0.198918,-0.225630,-0.208138,1.659291,-0.11847,-0.179284,-0.824762,-0.738549,1.768777,0.0
3,0.998307,0.022866,0.668434,0.177834,0.177834,1.344381,1.047605,1.627924,0.764545,2.552955,0.492606,-0.984507,-0.48967,0.0,-0.132686,-0.212846,1.170891,-0.352472,-0.725469,-0.058926,-1.060660,0.699779,-0.294906,1.374773,-0.371208,-1.068055,-0.208138,-0.321019,-0.732963,-0.257722,-0.11847,-0.198918,4.432026,-0.208138,-0.602667,-0.11847,-0.179284,-0.824762,1.354006,-0.565362,0.0
4,0.848810,-0.183363,1.081653,-0.445124,-0.445124,1.344381,0.896590,1.627924,2.055053,-0.406225,-0.686303,0.771805,-0.48967,0.0,-0.132686,0.491336,0.123990,-0.352472,-0.152326,-0.058926,0.942809,0.699779,-1.669549,-0.727393,2.693908,-1.068055,-0.208138,-0.321019,1.364326,-0.257722,-0.11847,-0.198918,-0.225630,-0.208138,-0.602667,-0.11847,-0.179284,-0.824762,-0.738549,1.768777,1.0


In [5]:
data.shape

(289, 41)

In [6]:
data.loan_status_bin.value_counts()

1.0    255
0.0     34
Name: loan_status_bin, dtype: int64

In [7]:
#Null Model (or random investment) will yield, 59.3% of picking a good loan: 
data.loan_status_bin.value_counts()[1] / len(data)

0.8823529411764706

In [8]:
X = data.drop('loan_status_bin', axis=1)
X.shape

(289, 40)

In [9]:
y = data['loan_status_bin']

In [10]:
n_features = 3
lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=n_features)

In [11]:
sfs.fit(X,y)

SequentialFeatureSelector(estimator=LogisticRegression(),
                          n_features_to_select=3)

In [12]:
#sfs.get_support()

In [13]:
print("Top {} features selected by forward sequential selection:{}"\
      .format(n_features, list(X.columns[sfs.get_support()])))


Top 3 features selected by forward sequential selection:['loan_amnt', 'annual_inc', 'dti']


In [14]:
feat = list(X.columns[sfs.get_support()])
feat

['loan_amnt', 'annual_inc', 'dti']

In [15]:
X_feat = data[feat]

In [16]:
y = data['loan_status_bin']
y

0      0.0
1      1.0
2      0.0
3      0.0
4      1.0
      ... 
284    1.0
285    1.0
286    1.0
287    1.0
288    1.0
Name: loan_status_bin, Length: 289, dtype: float64

In [17]:
lr_check = LogisticRegression()
steps = [('model', lr_check)]
pipeline = Pipeline(steps=steps)

In [18]:
#Accuracy:

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
scores = cross_val_score(pipeline, X_feat, y, scoring='accuracy', cv=cv, n_jobs=-1) 
print(scores)

score = np.mean(scores)
print(score)

[0.87931034 0.87931034 0.87931034 0.87931034 0.89473684]
0.8823956442831216


In [19]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
steps2 = [('model', ada)]
pipeline2 = Pipeline(steps=steps2)

In [20]:
#Accuracy:

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
scores = cross_val_score(pipeline2, X_feat, y, scoring='accuracy', cv=cv, n_jobs=-1) 
print(scores)

score = np.mean(scores)
print(score)

[0.84482759 0.81034483 0.79310345 0.86206897 0.85964912]
0.8339987900786447


In [21]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(alpha=1, max_iter=1000)
steps3 = [('model', nn)]
pipeline3 = Pipeline(steps=steps3)

In [22]:
#Accuracy:

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
scores = cross_val_score(pipeline3, X_feat, y, scoring='accuracy', cv=cv, n_jobs=-1) 
print(scores)

score = np.mean(scores)
print(score)

[0.87931034 0.87931034 0.87931034 0.87931034 0.89473684]
0.8823956442831216


In [23]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
steps4 = [('model', gb)]
pipeline4 = Pipeline(steps=steps4)

In [24]:
#Accuracy:

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
scores = cross_val_score(pipeline4, X_feat, y, scoring='accuracy', cv=cv, n_jobs=-1) 
print(scores)

score = np.mean(scores)
print(score)

[0.82758621 0.87931034 0.81034483 0.82758621 0.87719298]
0.8444041137326073
